### EXTRAÇÃO E TRATAMENTO DOS DADOS DE ENTIDADES SUPERVISIONADAS

- **Nome oficial da base:** IF.data
- **Órgão:** CMN, BCB
- **Periodicidade:** mensal 
- **Período:** 2011 a 2022 
- **Forma de extração:** API
- **Link:** https://dadosabertos.bcb.gov.br/dataset/dados-cadastrais-de-entidades-autorizadas

**Por Isabella de Moura Rocha**

Graduada em Administração, pela UFSCar Sorocaba e Mestra em Administração, pela UFSCar Sorocaba, isabellamrocha0@gmail.com

*Os dados foram usados na dissertação: ROCHA, I.M. Rentabilidade, eficiência e risco em cooperativas de crédito e bancos comerciais. 2023. Dissertação (Mestrado em Administração) – Universidade Federal de São Carlos, Sorocaba, 2023. Disponível em: https://repositorio.ufscar.br/handle/ufscar/18956.*

### 1. Bibliotecas necessárias

São importadas as bibliotecas necessárias para o processamento dos dados.

In [1]:
import pandas as pd
import requests

### 2. Requisição API BCB entidades supervisionadas

In [ ]:
entidades_supervisionadas = []

base_url = "https://olinda.bcb.gov.br/olinda/servico/BcBase/versao/v2/odata/EntidadesSupervisionadas"

inicio_ano = 2011 # quero esses anos de interesse, só alterar o de preferência
fim_ano = 2022

for year in range(inicio_ano, fim_ano + 1): # itera sobre todos os anos
    data_base = f"12-31-{year}"
    # Quero essas colunas de interesse. Quem quiser mais, só adicionar conforme a documentação.
    url = f"{base_url}(dataBase=@dataBase)?@dataBase='{data_base}'&$format=json" \
          "&$select=database,nomeEntidadeInteresseNaoFormatado,codigoCNPJ14,codigoCNPJ8," \
          "codigoTipoSituacaoPessoaJuridica,descricaoTipoSituacaoPessoaJuridica," \
          "codigoTipoEntidadeSupervisionada,descricaoTipoEntidadeSupervisionada,nomeReduzido"

    response = requests.get(url) # Faz a requisição

    if response.status_code == 200:
        data = response.json().get('value', [])
        if data:
            entidades_supervisionadas.extend(data)

cadastro_ifs = pd.DataFrame(entidades_supervisionadas)

In [ ]:
cadastro_ifs.to_parquet('EntidadesSupervisionadas_2011a2022.parquet')

### Filtrar entidades de interesse

Só estou interessada em Bancos Comerciais e Cooperativas de Crédito

In [7]:
cadastro_ifs = cadastro_ifs[(cadastro_ifs['descricaoTipoEntidadeSupervisionada']=='Banco Comercial')|
(cadastro_ifs['descricaoTipoEntidadeSupervisionada']=='Banco Comercial Estrangeiro - Filial no país')|
(cadastro_ifs['descricaoTipoEntidadeSupervisionada']=='Banco Múltiplo')|
(cadastro_ifs['descricaoTipoEntidadeSupervisionada']=='Cooperativa de Crédito')]

In [8]:
cadastro_ifs.to_parquet('dados_cadastrais_cooperativas_bancos_comerciais.parquet')